In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import wrds

In [3]:
temp = pd.read_csv('../../data/prices_data.csv')

In [4]:
temp2 = temp[['DlyCalDt', 'PERMNO', 'Ticker' ]]
temp2.drop_duplicates(inplace=True)
temp2.reset_index(drop=True, inplace=True)
permnos = temp2['PERMNO'].unique()

C:\Users\Asus\AppData\Local\Temp\ipykernel_16264\4189651190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2.drop_duplicates(inplace=True)


In [8]:
#temp2.to_csv('../../data/permno_ticker.csv', index=False)

In [11]:
# Establish connection to WRDS database
conn = wrds.Connection()

# Convert tuple to a comma-separated string
permno_str = ','.join(map(str, permnos))

# Specify query to retrieve adjusted daily prices from CRSP dataset
query_prices = f"""
        SELECT dsf.date, dsf.permno, dsf.prc, dsf.cfacpr, dsf.cfacshr
        FROM crsp.dsf
        WHERE dsf.date BETWEEN '2008-01-01' AND '2022-01-01'
        AND dsf.permno IN ({permno_str})
        """

# Specify query to retrieve dividends from CRSP dataset
query_dividends = f"""
        SELECT date, permno, divamt
        FROM crsp.dse
        WHERE date BETWEEN '2008-01-01' AND '2022-01-01'
        AND permno IN ({permno_str})
        """

# Execute queries and fetch data
prices_data = conn.raw_sql(query_prices)
dividends_data = conn.raw_sql(query_dividends)

# Merge price and dividend data
data = pd.merge(prices_data, dividends_data, on=['date', 'permno'], how='left')

# Calculate adjusted prices
data['adj_prc'] = abs(data['prc']) / data['cfacpr']
data['divamt'].fillna(0, inplace=True)
data['adj_prc_w_dividend'] = data['adj_prc'] + data['divamt']/data['cfacpr']

# Close connection
conn.close()

# Print adjusted prices data
print(data)

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
               date  permno    prc  cfacpr  cfacshr  divamt  adj_prc  \
0        2008-01-02   10299  30.95     1.0      1.0     0.0    30.95   
1        2008-01-02   10137  62.95     1.0      1.0     0.0    62.95   
2        2008-01-03   10137  63.47     1.0      1.0     0.0    63.47   
3        2008-01-04   10137  62.64     1.0      1.0     0.0    62.64   
4        2008-01-07   10137  63.06     1.0      1.0     0.0    63.06   
...             ...     ...    ...     ...      ...     ...      ...   
2406668  2011-07-12   93422  40.23     1.0      1.0     0.0    40.23   
2406669  2011-07-13   93422  40.29     1.0      1.0     0.0    40.29   
2406670  2011-07-14   93422  39.95     1.0      1.0     0.0    39.95   
2406671  2011-07-15   93422  41.53     1.0      1.0     0.0    41.53   
2406672  2011-0

C:\Users\Asus\AppData\Local\Temp\ipykernel_16264\2849516189.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['divamt'].fillna(0, inplace=True)


In [24]:
temp2['DlyCalDt'] = pd.to_datetime(temp2['DlyCalDt'])
data['date'] = pd.to_datetime(data['date'])

C:\Users\Asus\AppData\Local\Temp\ipykernel_16264\2074521110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2['DlyCalDt'] = pd.to_datetime(temp2['DlyCalDt'])


In [27]:
display(data)
display(temp2[temp2['PERMNO'] == 10299])

,date,permno,prc,cfacpr,cfacshr,divamt,adj_prc,adj_prc_w_dividend
0,2008-01-02,10299,30.95,1.0,1.0,0.0,30.95,30.95
1,2008-01-02,10137,62.95,1.0,1.0,0.0,62.95,62.95
2,2008-01-03,10137,63.47,1.0,1.0,0.0,63.47,63.47
3,2008-01-04,10137,62.64,1.0,1.0,0.0,62.64,62.64
4,2008-01-07,10137,63.06,1.0,1.0,0.0,63.06,63.06
...,...,...,...,...,...,...,...,...
2406668,2011-07-12,93422,40.23,1.0,1.0,0.0,40.23,40.23
2406669,2011-07-13,93422,40.29,1.0,1.0,0.0,40.29,40.29
2406670,2011-07-14,93422,39.95,1.0,1.0,0.0,39.95,39.95
2406671,2011-07-15,93422,41.53,1.0,1.0,0.0,41.53,41.53


,DlyCalDt,PERMNO,Ticker
37287,2000-01-03,10299,LLTC
37288,2000-01-04,10299,LLTC
37289,2000-01-05,10299,LLTC
37290,2000-01-06,10299,LLTC
37291,2000-01-07,10299,LLTC
...,...,...,...
41607,2017-03-07,10299,LLTC
41608,2017-03-08,10299,LLTC
41609,2017-03-09,10299,LLTC
41610,2017-03-10,10299,LLTC


In [29]:
# add ticker to data
data2 = pd.merge(data, temp2, left_on=['permno', 'date'], right_on=['PERMNO', 'DlyCalDt'], how='left')
data2.drop(columns=['PERMNO', 'DlyCalDt'], inplace=True)
data2

,date,permno,prc,cfacpr,cfacshr,divamt,adj_prc,adj_prc_w_dividend,Ticker
0,2008-01-02,10299,30.95,1.0,1.0,0.0,30.95,30.95,LLTC
1,2008-01-02,10137,62.95,1.0,1.0,0.0,62.95,62.95,AYE
2,2008-01-03,10137,63.47,1.0,1.0,0.0,63.47,63.47,AYE
3,2008-01-04,10137,62.64,1.0,1.0,0.0,62.64,62.64,AYE
4,2008-01-07,10137,63.06,1.0,1.0,0.0,63.06,63.06,AYE
...,...,...,...,...,...,...,...,...,...
2406668,2011-07-12,93422,40.23,1.0,1.0,0.0,40.23,40.23,QEP
2406669,2011-07-13,93422,40.29,1.0,1.0,0.0,40.29,40.29,QEP
2406670,2011-07-14,93422,39.95,1.0,1.0,0.0,39.95,39.95,QEP
2406671,2011-07-15,93422,41.53,1.0,1.0,0.0,41.53,41.53,QEP


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(data2[data2['adj_prc'].isna()])

In [62]:
data2.dropna(subset=['adj_prc'], inplace=True)

In [71]:
data2.to_csv('../../data/backtest_prices.csv', index=False)